In [1]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def run_experiment(func):
    invalid_p_values = 0
    for _ in tqdm(range(1000000)):
        p_value = func()
        if p_value < 0.05:
            invalid_p_values += 1
    print("P_h0(p < 0.05) = ", invalid_p_values / 1000000)

##### Basic

In [3]:
def run_one_sample_fixed():
    # generate synthetic data
    mu = 0
    std = 1  # standard deviation
    x = np.random.normal(loc=mu, scale=1)

    # we want to the following hypotheses
    # H_0: mu = 0   vs.   H_1: mu != 0

    # construct test-statistic
    T = x

    # compute two-sided p-value
    cdf = norm.cdf(T, loc=0, scale=1)
    p_value = 2 * min(cdf, 1 - cdf)

    return p_value

In [16]:
run_experiment(run_one_sample_fixed)

100%|██████████| 1000000/1000000 [01:03<00:00, 15832.77it/s]

P_h0(p < 0.05) =  0.050221


##### Get Max and compute p_value

In [4]:
def run_one_sample_max():
    # generate synthetic data
    mu = 0
    std = 1  # standard deviation
    x = np.random.normal(loc=mu, scale=std, size=3)
    T = np.max(x)

    # compute two-sided p-value
    cdf = norm.cdf(T, loc=mu, scale=std)
    p_value = 2 * min(cdf, 1 - cdf)

    return p_value

In [17]:
run_experiment(run_one_sample_max)

100%|██████████| 1000000/1000000 [01:02<00:00, 15931.71it/s]

P_h0(p < 0.05) =  0.073762


##### Get max and compute p_value with satisfaction T(xj | j was chosen)

In [11]:
def run_one_sample_max_has_chosen(n=3):
    mu = 0
    std = 1
    num_each_idx = [0] * n
    invalid_p_values = [0] * n
    invalid_p_value = 0
    for _ in tqdm(range(1000000)):
        x = np.random.normal(loc=mu, scale=std, size=n)
        max_idx = np.argmax(x)
        num_each_idx[max_idx] += 1
        T = x[max_idx]
        cdf = norm.cdf(T, loc=mu, scale=std)
        p_value = 2 * min(cdf, 1 - cdf)
        if p_value < 0.05:
            invalid_p_values[max_idx] += 1
            invalid_p_value += 1
    assert sum(num_each_idx) == 1000000
    for i in range(n):
        if num_each_idx[i] > 0:
            print("P_h0(p < 0.05 | {} was chosen) = ".format(i), (invalid_p_values[i] / num_each_idx[i])/((invalid_p_value / 1000000)*(num_each_idx[i] / 1000000)))
        else:
            print("P_h0(p < 0.05 | {} was chosen) = 0 (no occurrences)".format(i))
    
    

In [12]:
run_one_sample_max_has_chosen(n=3)

100%|██████████| 1000000/1000000 [01:01<00:00, 16274.84it/s]

P_h0(p < 0.05 | 0 was chosen) =  1.0010997068207135
P_h0(p < 0.05 | 1 was chosen) =  0.9996104308064362
P_h0(p < 0.05 | 2 was chosen) =  0.9992895713164998
